In [1]:
import os


### Change the label from 7 column to 6 and also change imange name

In [14]:
label_path_base = "/home/qimaqi/Downloads/datasets/amz/labels_amz"
label_wirte_path = "/home/qimaqi/Downloads/datasets/amz/labels"
val_txt_path = "/home/qimaqi/Downloads/datasets/amz/val.txt"
image_path = "./images/"
### open a label.txt
labels_list = os.listdir(label_path_base)
# print(labels_list)
val_f = open(val_txt_path,"w")
for file in labels_list:
    img_name = image_path + file[:-3] + "png" +"\n"
    val_f.write(img_name)
    lines_write = []
    f = open(os.path.join(label_wirte_path,file),"w")
    for line in open(os.path.join(label_path_base,file)):
        line_list = line.split(" ")
        line_list.pop(1)
        line_string_list = " ".join(line_list)
        # lines_write
        lines_write.append(line_string_list)
        # lines_write = [line_o+"/n" for line_o in lines_write]
    for line_i in lines_write:
        f.write(line_i)
    
    # f.writelines(lines_write)
    f.close()


In [ ]:
import os

label_path_base = "/home/qimaqi/Downloads/datasets/amz/labels_amz"
label_wirte_path = "/home/qimaqi/Downloads/datasets/amz/labels"

image_path = "./images/"
### open a label.txt
labels_list = os.listdir(label_path_base)
# print(labels_list)
for file in labels_list:
    img_name = image_path + file[:-3] + "png" +"\n"
    lines_write = []
    f = open(os.path.join(label_wirte_path,file),"w")
    for line in open(os.path.join(label_path_base,file)):
        line_list = line.split(" ")
        line_list.pop(1)
        line_string_list = " ".join(line_list)
        # lines_write
        lines_write.append(line_string_list)
        # lines_write = [line_o+"/n" for line_o in lines_write]
    for line_i in lines_write:
        f.write(line_i)
    
    f.close()

### extract images

In [2]:
import rosbag
import numpy as np
import cv2
from cv_bridge import CvBridge
from tqdm import tqdm

rosbag_path = '/media/qimaqi/My Passport/tmp/amz/be_autox_2022-05-11-15-16-17.bag'
self_bag = rosbag.Bag(rosbag_path)
type_and_topic_info = self_bag.get_type_and_topic_info(
            topic_filters=None)

topic = '/sensors/forward_camera/image_color'
DATA_FOLDER = '/media/qimaqi/My Passport/tmp/amz/04-29/'

topic_name_to_folder_name_dict = {
            "/sensors/fw_lidar/point_cloud_raw": "fw_lidar",
            "/sensors/mrh_lidar/point_cloud_raw": "mrh_lidar",
            "/sensors/forward_camera/image_color": "forward_camera",
            "/perception/lidar/motion_compensated/merge_pc": "merge_pc",
            "/perception/sensor_fusion_overlay/point_cloud_compensated":"compensated_pc"
        }

pbar = tqdm(total=type_and_topic_info[1][topic].message_count,
            desc=topic)

data_dir = os.path.join(DATA_FOLDER,
                        topic_name_to_folder_name_dict[topic])
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

counter = 0
timestamps = []
images = []
bridge = CvBridge()

for _, msg, _ in self_bag.read_messages(topics=[topic]):

    timestamp = float("{}.{}".format(
        str(msg.header.stamp.secs),
        str(msg.header.stamp.nsecs).zfill(9)))
    
    timestamp_ns = msg.header.stamp.to_nsec()

    image = bridge.imgmsg_to_cv2(msg, 'bgr8')
    cv2.imwrite(os.path.join(data_dir,
                                str(timestamp_ns).zfill(25) + '.png'), image)
    timestamps.append(timestamp)
    images.append(image)

    counter += 1

    pbar.update(1)

pbar.close()


/sensors/forward_camera/image_color: 100%|██████████| 1466/1466 [04:35<00:00,  5.32it/s]
